In [403]:
from collections import defaultdict
from __future__ import annotations
from typing import List, Dict, Tuple
import random
import numpy as np

In [404]:
class Simplex:
    def __init__(self, vertices: List[int]):
        self.n = len(vertices)
        self.vertices = vertices

    def get_representation(self) -> List[int]:
        return sorted(self.vertices)

    def __eq__(self, other: Simplex) -> bool:
        return self.get_representation() == other.get_representation()

    def get_immediate_children(self) -> List[Simplex]:
        if self.n == 1:
            return []
        return [
            Simplex([*self.vertices[:i], *self.vertices[i + 1 :]])
            for i in range(self.n)
        ]

    def __str__(self) -> str:
        return f"Simplex({' '.join(str(v) for v in self.get_representation())})"

    def __repr__(self) -> str:
        return self.__str__()


a = Simplex([1, 2, 3])
print(a)
print(a.get_immediate_children())

Simplex(1 2 3)
[Simplex(2 3), Simplex(1 3), Simplex(1 2)]


In [405]:
class SimplicialComplex:
    def __init__(self, simplices: List[Simplex]):
        # It will be useful later to keep the simplices graded by dimension.
        self.graded_simplices = defaultdict(list)
        for simplex in simplices:
            self.graded_simplices[simplex.n].append(simplex)
        self.largest_n = max(self.graded_simplices.keys())
        self.fill_in_gaps()

    def fill_in_simplex(self, simplex: Simplex):
        if simplex in self.graded_simplices[simplex.n]:
            return
        self.graded_simplices[simplex.n].append(simplex)
        for child in simplex.get_immediate_children():
            self.fill_in_simplex(child)

    def fill_in_gaps(self):
        for n in range(1, self.largest_n + 1):
            for simplex in self.graded_simplices[n]:
                for child in simplex.get_immediate_children():
                    self.fill_in_simplex(child)

    def __str__(self) -> str:
        return f"SimplicialComplex({dict(self.graded_simplices)})"

    def __repr__(self) -> str:
        return self.__str__()


a, b = Simplex([1, 2, 3]), Simplex([2, 3])
X = SimplicialComplex([a, b])
print(X)

SimplicialComplex({3: [Simplex(1 2 3)], 2: [Simplex(2 3), Simplex(1 3), Simplex(1 2)], 1: [Simplex(3), Simplex(2), Simplex(1)]})


In [406]:
class CellularSheaf:
    def __init__(self, simplicial_complex: SimplicialComplex):
        """
        In this version of a cellular sheaf, we will assume that each simplex is assigned R,
        and that the boundary maps are real numbers.
        """

        self.simplicial_complex = simplicial_complex
        self.largest_n = simplicial_complex.largest_n

        # We check that the boundary condition holds
        self.check_valid_simplicial_complex()

    def check_valid_simplicial_complex(self):
        for n in range(1, self.largest_n + 1):
            for simplex in self.simplicial_complex.graded_simplices[n]:
                for child in simplex.get_immediate_children():
                    if child not in self.simplicial_complex.graded_simplices[n - 1]:
                        raise ValueError(
                            f"Simplex {child} is not in the simplicial complex"
                        )

    def generate_random_path_value_mapping(self) -> Dict[str, float]:
        path_value_map = {}
        for n in range(1, self.largest_n + 1):
            for simplex in self.simplicial_complex.graded_simplices[n]:
                # For now, set to be random number between 1 and 10
                # Since Simplex is a class we can't hash it, so we need to use the representation
                path_value_map[str(simplex)] = random.random() * 9 + 1
        return path_value_map

    def get_boundary_map_matrices(
        self, path_value_map: Dict[str, float]
    ) -> Dict[(int, int), List[List[float]]]:

        boundary_map_matrices = {}
        for a in range(1, self.largest_n):
            b = a + 1
            # Note that b is for the larger dimensional simplices
            input_dim, output_dim = (
                len(self.simplicial_complex.graded_simplices[b]),
                len(self.simplicial_complex.graded_simplices[a]),
            )
            boundary_map = np.zeros((input_dim, output_dim))
            for i in range(input_dim):
                current_simplex = self.simplicial_complex.graded_simplices[b][i]
                desired_path_value = path_value_map[str(current_simplex)]
                sign = 1
                for j in range(output_dim):
                    current_child = self.simplicial_complex.graded_simplices[a][j]
                    if current_child in current_simplex.get_immediate_children():
                        child_path_value = path_value_map[str(current_child)]
                        boundary_map[i, j] = (
                            sign * desired_path_value / child_path_value
                        )
                        sign *= -1
                    else:
                        boundary_map[i, j] = 0

            boundary_map_matrices[(b, a)] = boundary_map
        return boundary_map_matrices


print(X.graded_simplices)
C = CellularSheaf(X)
path_value_map = C.generate_random_path_value_mapping()
print(path_value_map)
boundary_map_matrices = C.get_boundary_map_matrices(path_value_map)
print(boundary_map_matrices)

print(boundary_map_matrices[(3, 2)] @ boundary_map_matrices[(2, 1)])

defaultdict(<class 'list'>, {3: [Simplex(1 2 3)], 2: [Simplex(2 3), Simplex(1 3), Simplex(1 2)], 1: [Simplex(3), Simplex(2), Simplex(1)]})
{'Simplex(3)': 5.9633235547581345, 'Simplex(2)': 6.378590579213085, 'Simplex(1)': 2.647014935588701, 'Simplex(2 3)': 9.25094285370236, 'Simplex(1 3)': 5.627548659169039, 'Simplex(1 2)': 2.668676628557633, 'Simplex(1 2 3)': 3.6152215536580226}
{(2, 1): array([[ 1.55130654, -1.45031143,  0.        ],
       [ 0.94369333,  0.        , -2.12599808],
       [ 0.        ,  0.4183803 , -1.00818344]]), (3, 2): array([[ 0.39079493, -0.64241498,  1.35468701]])}
[[ 5.49917743e-17  1.21337536e-17 -2.46774029e-16]]


In [407]:
a, b, c, d = Simplex([1, 2, 3]), Simplex([2, 3, 4]), Simplex([2, 3]), Simplex([1, 3])
X = SimplicialComplex([a, b, c, d])
Xc = CellularSheaf(X)
Y = SimplicialComplex([b, c, d])
Yc = CellularSheaf(Y)

print(X.graded_simplices)
print(Y.graded_simplices)

defaultdict(<class 'list'>, {3: [Simplex(1 2 3), Simplex(2 3 4)], 2: [Simplex(2 3), Simplex(1 3), Simplex(1 2), Simplex(3 4), Simplex(2 4)], 1: [Simplex(3), Simplex(2), Simplex(1), Simplex(4)]})
defaultdict(<class 'list'>, {3: [Simplex(2 3 4)], 2: [Simplex(2 3), Simplex(1 3), Simplex(3 4), Simplex(2 4)], 1: [Simplex(3), Simplex(2), Simplex(1), Simplex(4)]})


In [408]:
def elementary_swap(n, i, j):
    """ Create an elementary matrix that swaps rows i and j in an n x n matrix """
    E = np.eye(n)
    E[[i, j]] = E[[j, i]]
    return E

def elementary_scale(n, i, k):
    """ Create an elementary matrix that scales row i by k in an n x n matrix """
    E = np.eye(n)
    E[i, i] = k
    return E

def elementary_add(n, i, j, k):
    """ Create an elementary matrix that adds k times row j to row i in an n x n matrix """
    E = np.eye(n)
    E[i, j] = k
    return E

def rref_and_operations(A):
    m, n = A.shape
    A = A.astype(float)  # Use float to prevent integer division issues
    history = []  # To store history of operations as matrices

    for i in range(min(m, n)):
        # Find the pivot
        max_row = np.argmax(np.abs(A[i:, i])) + i
        if A[max_row, i] == 0:
            continue

        # Swap if necessary
        if max_row != i:
            E = elementary_swap(m, i, max_row)
            A = E @ A
            history.append(E)

        # Normalize the pivot row
        pivot = A[i, i]
        E = elementary_scale(m, i, 1 / pivot)
        A = E @ A
        history.append(E)

        # Eliminate down
        for k in range(m):
            if k != i and A[k, i] != 0:
                E = elementary_add(m, k, i, -A[k, i])
                A = E @ A
                history.append(E)

    return A, history


A = np.array([[1, 2, 3], [4, 5, 6]])
row_reduced, operations = rref_and_operations(A)
print(row_reduced)
product_of_operations = np.eye(2)
for operation in operations:
    product_of_operations = operation @ product_of_operations
recovered_row_reduced = np.round(product_of_operations @ A)
print(recovered_row_reduced)

[[ 1.  0. -1.]
 [ 0.  1.  2.]]
[[ 1.  0. -1.]
 [ 0.  1.  2.]]


In [409]:
def get_decomposition(A):
    row_reduced, operations = rref_and_operations(A)
    product_of_operations = np.eye(A.shape[0])
    for operation in operations:
        product_of_operations = operation @ product_of_operations
    return product_of_operations, row_reduced

In [410]:
from sympy import *

k = 2

# Inner product matrices set to identity for now
B_Y_k = Yc.get_boundary_map_matrices(Yc.generate_random_path_value_mapping())[
    (k, k - 1)
]
B_X_k = Xc.get_boundary_map_matrices(Xc.generate_random_path_value_mapping())[
    (k, k - 1)
]
if (k+1, k) in Yc.get_boundary_map_matrices(Yc.generate_random_path_value_mapping()):
    B_Y_kplusone = Yc.get_boundary_map_matrices(Yc.generate_random_path_value_mapping())[
        (k + 1, k)
    ]
else:
    B_Y_kplusone = np.zeros((len(Y.graded_simplices[k + 1]), len(Y.graded_simplices[k])))
B_X_kplusone = Xc.get_boundary_map_matrices(Xc.generate_random_path_value_mapping())[
    (k + 1, k)
]

down_laplacian = B_Y_k @ B_Y_k.T

n_X_k = len(X.graded_simplices[k])
n_Y_k = len(Y.graded_simplices[k])
if n_X_k == n_Y_k:
    B_XY_kplusone = B_X_kplusone
    up_laplacian = B_XY_kplusone.T @ B_XY_kplusone
else:
    columns_to_remove = n_Y_k
    D_X_kplusone = B_X_kplusone[:, columns_to_remove:]
    # Round to avoid numerical errors
    D_X_kplusone = np.round(D_X_kplusone, 1)
    
    Y, R_X_kplusone = get_decomposition(D_X_kplusone)
    R_X_kplusone = R_X_kplusone.T
    zero_column_indices = np.where(~R_X_kplusone.any(axis=0))[0]
    
    if zero_column_indices.size == 0:
        up_laplacian = np.zeros((n_Y_k, n_Y_k))
    else:
        Z = Y[:, zero_column_indices]
        print(Y.shape)
        B_XY_kplusone = B_Y_kplusone @ Y[:n_Y_k, zero_column_indices]
        print(B_XY_kplusone.shape)
        up_laplacian = B_XY_kplusone @ np.linalg.inv(Z.T @ Z) @ B_XY_kplusone.T

print(up_laplacian)

(2, 2)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)

TODO:
- Tests for the generated boundary maps
- Random cellular complex generation